# **Survey of Consumer Finances Analysis**
# Working Title: Perceptions and Reality of Household Finance Risk
## Dan Valenzuela

***



## **Overview** <a id="Overview"></a>

[**1. Business Problem**](#Business-Problem)

[**2. Data Understanding**](#Data-Understanding)

[**3. Data Preparation**](#Data-Preparation)

[**4. Data Analysis**](#Data-Analysis)

[**5. Evaluation**](#Evaluation)

[**6. Conclusion & Next Steps**](#Conclusion)

[**7. Endnotes**](#Endnotes)

***

## **Business Problem** <a id="Business-Understanding"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Data Understanding** <a id="Data-Understanding"></a>
[*↑ Back to overview*](#Overview)

### Datasets

The data can be summarized by the [Federal Reserve](https://www.federalreserve.gov/publications/files/scf20.pdf) itself [<sup>1</sup>](#fn-1)<a id='fn-1-s'></a>: 

> The SCF is a triennial interview survey of U.S. families sponsored by the
Board of Governors of the Federal Reserve System with the cooperation of the U.S.
Department of the Treasury. Since 1992, data for the SCF have been collected by NORC, a
research organization at the University of Chicago. Although the majority of the data are
collected between May and December of each survey year, a small fraction of the data
collection occurs in the first four months of the next calendar year. In the 2019 SCF, this
portion of the data collection overlapped with early months of the COVID-19 pandemic,
with about 9 percent of interviews conducted between February and April 2020.

 

### Sampling
From [Federal Reserve Bulletin 2020 at 40](https://www.federalreserve.gov/publications/files/scf20.pdf)
>First, a standard multistage area-probability sample (a geographically based random sample) is selected
to provide good coverage of characteristics, such as homeownership, that are broadly
distributed in the population.
Second, a supplemental sample is selected to disproportionately include wealthy families,
which hold a relatively large share of such thinly held assets as noncorporate businesses
and tax-exempt bonds. Called the “list sample,” this group is drawn from a list of statistical
records derived from tax returns. These records are used under strict rules governing
confidentiality, the rights of potential respondents to refuse participation in the survey, and
the types of information that can be made available. Persons listed by Forbes as being
among the wealthiest 400 people in the United States are excluded from sampling.

>Of the 5,783 interviews completed for the 2019 SCF, 4,291 were from the area-probability
sample, and 1,492 were from the list sample; for 2016, 4,754 were from the area-probability
sample, and 1,500 were from the list sample.

### Weighting

From [Federal Reserve Bulletin 2020 at 42](https://www.federalreserve.gov/publications/files/scf20.pdf)

>To provide a measure of the frequency with which families similar to the sample families
could be expected to be found in the population of all families, an analysis weight is
computed for each case, accounting for both the systematic properties of the sample design
and differential patterns of nonresponse. The SCF response rates are low by the standards
of some other major government surveys, and analysis of the data confirms that the
tendency to refuse participation is highly correlated with net worth. However, unlike other
surveys, which almost certainly also have differential nonresponse by wealthy households,
the SCF has the means to adjust for such nonresponse.

### Imputation

From [Codebook 2019](https://www.federalreserve.gov/econres/files/codebk2019.txt)

> The imputations are stored as five successive replicates
("implicates") of each data record.  Thus, the number of observations
in the full data set (28,915) is five times the actual number of
respondents (5,783)

### Key Data for Merging Datasets and Analysis <a id="Key-Data" ></a>



[*↑ Back to overview*](#Overview)
***

## **Data Preparation** <a id="Data-Preparation"></a>
[*↑ Back to overview*](#Overview)

### Merging Data





### Cleaning Data <a id="Cleaning-Data"></a>



[*↑ Back to overview*](#Overview)
***

## **Data Analysis** <a id="Data-Analysis"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Evaluation**<a id="Evaluation"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Conclusion & Next Steps**<a id="Conclusion"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Endnotes** <a id="Endnotes"></a>
[*↑ Back to overview*](#Overview)


[1.](#fn-1-s)<a id='fn-1'></a> This is footnote 1

[*↑ Back to overview*](#Overview)